In [1]:
#Needed installs
#%pip install scikit-learn

In [2]:
import torch

# Check if CUDA is available
print("Is CUDA available:", torch.cuda.is_available())

# If you have multiple GPUs, check how many are available
print("Number of GPUs available:", torch.cuda.device_count())

# Get the name of the current GPU device
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))


# Prints the CUDA version PyTorch was built with
print("PyTorch built with CUDA Version:", torch.version.cuda)

Is CUDA available: True
Number of GPUs available: 1
GPU Name: NVIDIA GeForce RTX 4090 Laptop GPU
PyTorch built with CUDA Version: 12.4


In [3]:
import os,sys
from torch.utils.data import DataLoader
import torch

notebook_path=os.getcwd()
sys.path.append(os.path.abspath(os.path.join(notebook_path,'NN_Framework')))
from NN_Framework import model_utils
from NN_Framework.mibi_dataset import MibiDataset
from NN_Framework.mibi_vit import ViTBinaryClassifier

In [4]:
data_path=r'D:\MIBI-TOFF\Data_For_Amos'
train_dataset=MibiDataset(hdf5_path=r'D:\MIBI-TOFF\Scratch\training.h5')
train_loader=DataLoader(dataset=train_dataset,batch_size=50,shuffle=True, num_workers=4)

val_dataset=MibiDataset(hdf5_path=r'D:\MIBI-TOFF\Scratch\validation.h5')
val_loader=DataLoader(dataset=train_dataset,batch_size=50,shuffle=True, num_workers=4)

val_dataset=MibiDataset(hdf5_path=r'D:\MIBI-TOFF\Scratch\testing.h5')
val_loader=DataLoader(dataset=train_dataset,batch_size=50,shuffle=True, num_workers=4)

In [8]:
dims_scaling=2
model = ViTBinaryClassifier(
    img_size_x=128, img_size_y=128, in_channels=39, num_classes=2, 
    patch_size_x=16, patch_size_y=16, embed_dim=768*dims_scaling, num_heads=12, 
    depth=12, mlp_dim=768*dims_scaling*4, dropout_rate=0.1, weight_decay=1e-5
)
criterion = torch.nn.CrossEntropyLoss()  # Define the classification criterion
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4,weight_decay=1e-5)  # Define the optimizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # Set device to GPU (0)
model.to(device)
print(torch.cuda.is_available(),device)
print(next(model.parameters()).device) 

True cuda:0
cuda:0


In [6]:
print(train_dataset.class_counts)

{0: 14592, 1: 15680}


In [9]:

import mlflow
mlflow.end_run()

model_utils.train_model(model, train_loader, val_loader, criterion, optimizer, device, location=r'D:\MIBI-TOFF\Scratch\DL_Results', 
    epochs=50, patience=25, delta=0.00000001, check_val_freq=5,num_classes=2, log_with_mlflow=True,mlflow_uri="http://127.0.0.1:5000")

2024/10/23 17:05:19 INFO mlflow.tracking._tracking_service.client: 🏃 View run unequaled-turtle-915 at: http://127.0.0.1:5000/#/experiments/0/runs/9411fca2190c4628aeb6b5ef7faefe2d.
2024/10/23 17:05:19 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/0.
